## 예제로 배우는 파이토치(Pytorch)

### 특징

- Numpy와 유사하지만 GPU 상에서 실행 가능한 N차원 Tensor
- 신경망을 구성하고 학습하는 과정에서의 자동 미분

## Data Matrix

<center>

$ X = \begin{bmatrix} x_{11} &...& x_{1p} \\ & \vdots & \\ x
_{n1}&  ... & x_{np}  \end{bmatrix}  $ : $n \times p$

<br>
$ Y = \begin{bmatrix} y_{11} &...& y_{1k} \\ & \vdots & \\ y
_{n1}&  ... & y_{nk}  \end{bmatrix}  $ : $n \times k$

where  $y_{ij} \in \{ 0,1 \} $ and $\sum_{j=1}^k y_{ij} = 1$

<br>

$ W_1 = \begin{bmatrix} w_{111} & w_{112} &...& w_{11h} \\ & & \vdots & \\ w
_{1p1}& w_{1p2} & ... & w_{1ph}  \end{bmatrix}  $ : $p \times h$


<br>

$ W_2 = \begin{bmatrix} w_{211} & w_{212} &...& w_{21k} \\& & \vdots & \\ w_{2h1}& w_{2h2} & ... & w_{2hk}  \end{bmatrix}  $ : $h \times k$

</center>


<br>

## Feed Forward

<center>
<br>

$ H = X \cdot W_1$ : $n \times h$

$ H_{relu} = ReLU(H) $ : $n \times h$

$ Y_{pred} = H_{relu} \cdot W_2 $ : $n \times k$

</center>

<br>

## Back Propagation

<center>

$ l(W_1, W_2) =  Tr[(Y_{pred} - Y )^T (Y_{pred} - Y)] = \sum_{i=1}^{n} \sum_{j=1}^{k} (y_{ij}^{pred} -y_{ij} )^2   $

</center>

<br>

<center>

$W_{2}^{(t)} = W_{2}^{(t-1)} + \eta {\partial l(W_1, W_2) \over \partial W_{2} }$

${\partial l(W_1, W_2) \over \partial W_{2} } = [ {\partial Y_{pred} \over \partial W_{2}} ]^T \cdot {\partial l(W_1, W_2) \over \partial Y_{pred}}  = H_{relu}^T \cdot 2(Y_{pred} - Y) $ : grad_w2


<br>

$W_{1}^{(t)} = W_{1}^{(t-1)} + \eta {\partial l(W_1, W_2) \over \partial W_{1} }$

${\partial l(W_1, W_2) \over \partial W_1} =  [{\partial H \over \partial W_1}]^T \cdot  { H_{relu} \over \partial H} ( {\partial l(W_1, W_2) \over \partial Y_{pred}} \cdot [{\partial Y_{pred}  \over \partial H_{relu}}]^T ) = [{\partial H \over \partial W_1}]^T \cdot  { H_{relu} \over \partial H}  $ (grad_h_relu)

= $[{\partial H \over \partial W_1}]^T \cdot $ grad_h

</center>

# Numpy

In [ ]:
import numpy as np

# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.


N, D_in, H, D_out = 64,1000,100,10

# 무작위의 입력과 출력 데이터를 생성합니다

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 무작위로 가중치를 초기화합니다.

w1 = np.random.randn(D_in , H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

for t in range(500):
  # 순전파 단계 : 예측값 y를 계산합니다.
  h = x.dot(w1) # N * H
  h_relu = np.maximum(h,0) # N*H
  y_pred = h_relu.dot(w2) # N*D_out

  # 손실(loss)을 계산하고 출력합니다.
  loss = np.square(y_pred - y).sum() # N * D_out
  if t % 100 == 99:
    print(t, loss)

  # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다.
  grad_y_pred = 2.0 * (y_pred - y)  # N * D_out
  grad_w2 = h_relu.T.dot(grad_y_pred) # H * D_out
  grad_h_relu = grad_y_pred.dot(w2.T) # N * H
  grad_h = grad_h_relu.copy() # N * H
  grad_h[h < 0] = 0
  grad_w1 = x.T.dot(grad_h) # D_in * H

  # 가중치를 갱신합니다.
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

99 465.5093770879764
199 1.6511792363910374
299 0.00876622884089416
399 5.146973608674881e-05
499 3.1382471053474595e-07


# PyTorch: Tensors

In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # GPU에서 실행하려면 이 주석을 제거하세요.

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성합니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 무작위로 가중치를 초기화합니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 손실(loss)을 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신합니다.
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 318.3465576171875
199 0.8928102850914001
299 0.004582075402140617
399 0.0001473629818065092
499 3.298965748399496e-05


# tensorflow : ?????

In [ ]:
import tensorflow as tf
from tensorflow import keras

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성합니다.
x = tf.random.normal([N, D_in], dtype=tf.float32)
y = tf.random.normal([N, D_out], dtype=tf.float32)

# 무작위로 가중치를 초기화합니다.
w1 = tf.Variable(tf.random.normal([D_in, H]), dtype=tf.float32)
w2 = tf.Variable(tf.random.normal([H, D_out]), dtype=tf.float32)

learning_rate = 1e-2

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(lr = learning_rate)

train_loss = tf.keras.metrics.Mean()

with tf.GradientTape() as tape:
    # 순전파 단계: 예측값 y를 계산합니다.
    h = tf.matmul(x,w1)
    h_relu = tf.maximum(h,0)
    y_pred = tf.matmul(h_relu,w2)
    # loss function
    loss = loss_object(y_pred, y)  
gradient = tape.gradient(loss, [w1,w2])
optimizer.apply_gradients(zip(gradient, [w1,w2]))
  

# 반복 훈련
for i in range(500):
  if i % 10 == 1:
    print("스텝 {:02d}에서 손실: {:.3f}".format(i, loss))

스텝 01에서 손실: 52388.297
스텝 11에서 손실: 52388.297
스텝 21에서 손실: 52388.297
스텝 31에서 손실: 52388.297
스텝 41에서 손실: 52388.297
스텝 51에서 손실: 52388.297
스텝 61에서 손실: 52388.297
스텝 71에서 손실: 52388.297
스텝 81에서 손실: 52388.297
스텝 91에서 손실: 52388.297
스텝 101에서 손실: 52388.297
스텝 111에서 손실: 52388.297
스텝 121에서 손실: 52388.297
스텝 131에서 손실: 52388.297
스텝 141에서 손실: 52388.297
스텝 151에서 손실: 52388.297
스텝 161에서 손실: 52388.297
스텝 171에서 손실: 52388.297
스텝 181에서 손실: 52388.297
스텝 191에서 손실: 52388.297
스텝 201에서 손실: 52388.297
스텝 211에서 손실: 52388.297
스텝 221에서 손실: 52388.297
스텝 231에서 손실: 52388.297
스텝 241에서 손실: 52388.297
스텝 251에서 손실: 52388.297
스텝 261에서 손실: 52388.297
스텝 271에서 손실: 52388.297
스텝 281에서 손실: 52388.297
스텝 291에서 손실: 52388.297
스텝 301에서 손실: 52388.297
스텝 311에서 손실: 52388.297
스텝 321에서 손실: 52388.297
스텝 331에서 손실: 52388.297
스텝 341에서 손실: 52388.297
스텝 351에서 손실: 52388.297
스텝 361에서 손실: 52388.297
스텝 371에서 손실: 52388.297
스텝 381에서 손실: 52388.297
스텝 391에서 손실: 52388.297
스텝 401에서 손실: 52388.297
스텝 411에서 손실: 52388.297
스텝 421에서 손실: 52388.297
스텝 431에서 손실: 52388.29